In [1]:
import ale_py
import gym
import gym
from stable_baselines3 import PPO
import base64

from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack

# for rendering
import imageio
import IPython
from PIL import Image
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [2]:
# There already exists an environment generator that will make and wrap atari environments correctly.
env_name = "ALE/FishingDerby-v5"
env = make_atari_env(env_name, n_envs=4, seed=0)
# Stack 4 frames
env = VecFrameStack(env, n_stack=4)

# gym 
# env = gym.make('ALE/FishingDerby-v5')
# model = PPO("MlpPolicy", env, verbose=0, tensorboard_log="./tensorboards/")
# model.learn(total_timesteps=20000)

In [3]:
pretrained_model = A2C('CnnPolicy', env, verbose=0, tensorboard_log="./tensorboards/")
pretrained_model.learn(total_timesteps=20000)

In [4]:
pretrained_model.save("a2c_fishing")
trained_model = A2C.load("a2c_fishing", verbose=0, tensorboard_log="./tensorboards/")
env = make_atari_env(env_name, n_envs=4, seed=0)
env = VecFrameStack(env, n_stack=4)
trained_model.set_env(env)

In [5]:
trained_model.learn(total_timesteps=int(0.5e5))

In [6]:
def create_policy_eval_video(eval_env, policy_model, filename="eval_video", num_episodes=1000, fps=30):
    filename = filename + ".gif"
    with imageio.get_writer(filename, fps=fps) as video:
        obs = eval_env.reset()
        for _ in range(num_episodes):
            action, _states = policy_model.predict(obs, deterministic=True)
            obs, reward, done, info = eval_env.step(action)
            im = eval_env.render(mode="rgb_array")
            video.append_data(im)
            if done.all():
                obs = eval_env.reset()
                # video.append_data(eval_env.render(mode=mode))
                break
        eval_env.close()
    return filename
    
create_policy_eval_video(env, trained_model)

'eval_video.gif'